In [1]:
from flask import Flask, render_template, request

import pandas as pd
import gensim
import spacy
nlp = spacy.load("en_core_web_sm")
from joblib import load


from gensim.models import TfidfModel
from bs4 import BeautifulSoup

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexandremonod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandremonod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexandremonod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/alexandremonod/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
def suppr_balises_html(text):
    soup = BeautifulSoup(text, "html.parser")
    text_out = ' '.join(soup.stripped_strings)
    # Remove \n
    text_out = text_out.replace("\n", " ")        
    return text_out

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def replace_pos(tokens):
    k = 0
    for i in tokens.keys():
        tokens[i] = get_wordnet_pos(tokens[i])
        k += 1
    return tokens

def lemmatize(tokens):
    WNlemmatizer = WordNetLemmatizer()
    lem_tokens = []
    for key in tokens.keys():
        if tokens[key] is None :    # In case there are no tags
            lem_tokens.append(key)
        else :
            lem_tokens.append(WNlemmatizer.lemmatize(key, pos=tokens[key]) )
    return lem_tokens

def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele + ' '
    return str1

def keep_nouns(sentence):
    liste = []
    for word in sentence:
        if word.pos_ == "NOUN" or word.pos_ == "PROPN":
            liste.append(word)
    return liste

def tokenization_full(text):
    text_out = suppr_balises_html(text)
    text_out = text_out.lower()
    # Tokenize
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    tokenized = tokenizer.tokenize(text_out)
    # Stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    text_without_sw = [word for word in tokenized if word not in stopwords and len(word) > 2]
    # First need to POS-tag tokens
    postagged = dict(nltk.pos_tag(text_without_sw))
    tokens = replace_pos(postagged)
    lem_tokens = lemmatize(tokens)
    # NER
    tokens_from_list_to_strings = listToString(lem_tokens)
    sentences_nlp_ = nlp(tokens_from_list_to_strings)
    ner_tokens = keep_nouns(sentences_nlp_)
    return ner_tokens

In [2]:
# Chargement du modèle nécessaire à l'API
best_model = load('best_model')
model_fit = load('best_model_fit.pkl')

In [8]:
post_in = "<p>Is it possible to define a class in C# such that</p>\n\n<pre><code>class GenericCollection&lt;T&gt; : SomeBaseCollection&lt;T&gt; where T : Delegate\n</code></pre>\n\n<p>I couldn't for the life of me accomplish this last night in .NET 3.5. I tried using</p>\n\n<p><code>delegate, Delegate, Action&lt;T&gt; and Func&lt;T, T&gt;</code></p>\n\n<p>It seems to me that this should be allowable in some way. I'm trying to implement my own EventQueue.</p>\n\n<p>I ended up just doing this [primitive approximation mind you].</p>\n\n<pre><code>internal delegate void DWork();\n\nclass EventQueue {\n    private Queue&lt;DWork&gt; eventq;\n}\n</code></pre>\n\n<p>But then I lose the ability to reuse the same definition for different types of functions.</p>\n\n<p>Thoughts?</p>\n"

In [9]:
# Transformation des données saisies en listes de mots (mêmes transformations que notebook d'anal. explo.)
post_out = tokenization_full(post_in)
# post_out = [post_out]

In [10]:
# Model
cvect = CountVectorizer(stop_words='english', preprocessor=' '.join)
cv_fit = model_fit
cv_transform = cvect.transform(post_out)
ft_cv = cv_fit.get_feature_names_out()
cv_data = pd.DataFrame.sparse.from_spmatrix(cv_transform, columns=ft_cv)
#Prediction
prediction_cv_format = best_model.predict(cv_data)
multilabel_binarizer = MultiLabelBinarizer()
prediction_tag  = multilabel_binarizer.inverse_transform(prediction_cv_format)
print("Predicted:", prediction_tag[0:5]) # CHANGE WITH NAME OF THE COLUMNS WITH THE TAGS!
print("True:", df[0:5])
# Affichage du template de résultat
return render_template('resultat.html', Post=post_in,
                       Tags=prediction_tag[0:5])

NotFittedError: Vocabulary not fitted or provided